In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
data = pd.read_csv('spam.csv',encoding='latin-1')

data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":'label', "v2":'text'})
print(data.head())
tags = data["label"]
texts = data["text"]

  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


### To do: develop an accurate simple neural network model for spam classification (no LSTM, CNN, etc.)

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [3]:
data.isna().sum()

label    0
text     0
dtype: int64

In [4]:
data['label'].unique()
label = {'ham': 0, 'spam': 1}
data['label'].replace(label, inplace=True)
data.head()

,label,text
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
message = data['text'].tolist()
labels = data['label'].tolist()

In [6]:
#setting testing and training message and labels

training_size = int(len(message) * 0.8)

train_message = message[0:training_size]
test_message = message[training_size:]
train_labels = labels[0:training_size]
test_labels = labels[training_size:]

In [7]:
train_labels2 = np.array(train_labels)
test_labels2 = np.array(test_labels)

In [8]:
train_labels2.sum()

602

In [9]:
test_labels2.sum()

145

In [10]:
#dataset tokenizing

vocab_size = 1000
embedding_dim = 16
max_length = 100
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [11]:
#dataset tokenizing

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(train_message)
word_index = tokenizer.word_index
proc = tokenizer.texts_to_sequences(train_message)
pad = pad_sequences(proc, maxlen = max_length, padding = padding_type, 
                       truncating = trunc_type)

In [12]:
#dataset tokenizing

test_proc = tokenizer.texts_to_sequences(test_message)
test_pad = pad_sequences(test_proc,maxlen=max_length, 
                               padding=padding_type, truncating=trunc_type)

In [13]:
test_proc

[[975, 2, 1, 1, 143, 119, 2, 1, 22, 436, 10, 143, 1, 1, 1, 2, 25, 602],
 [865,
  3,
  209,
  107,
  606,
  41,
  119,
  9,
  49,
  1,
  4,
  49,
  565,
  3,
  1,
  1,
  1,
  1,
  28,
  182,
  1,
  3,
  1,
  69,
  53,
  93,
  3,
  1],
 [41,
  9,
  1,
  4,
  5,
  130,
  62,
  1,
  261,
  11,
  80,
  14,
  337,
  8,
  72,
  221,
  2,
  67,
  1,
  4,
  337,
  25,
  1,
  3,
  53,
  3,
  130,
  1,
  19,
  12,
  1,
  8,
  64,
  1,
  12,
  1,
  9,
  264,
  718,
  166,
  20,
  2,
  34,
  219,
  10,
  649,
  135,
  81],
 [179,
  199,
  13,
  14,
  88,
  87,
  45,
  9,
  36,
  1,
  292,
  10,
  8,
  83,
  7,
  84,
  1,
  6,
  1,
  8,
  1,
  12,
  696,
  268,
  23,
  24,
  454,
  1,
  879,
  184,
  127,
  237,
  1,
  1,
  25,
  2,
  160,
  82,
  3,
  106,
  80,
  48,
  3,
  140,
  12,
  1,
  224,
  111,
  1],
 [81, 2, 1, 188, 123, 1, 1, 630, 233, 37, 12, 1, 55, 4, 44, 3, 263],
 [71,
  153,
  2,
  397,
  870,
  523,
  326,
  559,
  141,
  26,
  2,
  260,
  426,
  1,
  392,
  1,
  46,
  11,
  498],


In [14]:
test_pad

array([[975,   2,   1, ...,   0,   0,   0],
       [865,   3, 209, ...,   0,   0,   0],
       [ 41,   9,   1, ...,   0,   0,   0],
       ...,
       [  1,  67,  10, ...,   0,   0,   0],
       [  6, 601, 109, ...,   0,   0,   0],
       [  1,  64, 546, ...,   0,   0,   0]])

In [15]:
word_index_reverse = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([word_index_reverse.get(i, '?') for i in text])

In [16]:
print(decode_review(pad[1]))

ok lar <OOV> wif u <OOV> ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?


In [17]:
print(train_message[1])

Ok lar... Joking wif u oni...


In [18]:
#Model
#training

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')])


In [19]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
history=model.fit(pad, train_labels2, epochs = 30, validation_data=(test_pad, test_labels2))
model.summary()

Epoch 1/30
140/140 [==============================] - 2s 5ms/step - loss: 0.3685 - accuracy: 0.8622 - val_loss: 0.2281 - val_accuracy: 0.8691
Epoch 2/30
140/140 [==============================] - 0s 3ms/step - loss: 0.1880 - accuracy: 0.8865 - val_loss: 0.1571 - val_accuracy: 0.9677
Epoch 3/30
140/140 [==============================] - 0s 3ms/step - loss: 0.1482 - accuracy: 0.9675 - val_loss: 0.1378 - val_accuracy: 0.9785
Epoch 4/30
140/140 [==============================] - 0s 3ms/step - loss: 0.1273 - accuracy: 0.9782 - val_loss: 0.1241 - val_accuracy: 0.9803
Epoch 5/30
140/140 [==============================] - 0s 3ms/step - loss: 0.1128 - accuracy: 0.9834 - val_loss: 0.1157 - val_accuracy: 0.9812
Epoch 6/30
140/140 [==============================] - 0s 3ms/step - loss: 0.1011 - accuracy: 0.9874 - val_loss: 0.1094 - val_accuracy: 0.9830
Epoch 7/30
140/140 [==============================] - 0s 3ms/step - loss: 0.0909 - accuracy: 0.9890 - val_loss: 0.1040 - val_accuracy: 0.9830
Epoch 

In [20]:
#using model to predict spam and creating sequence

recieved_text = data['text'].astype(str)
print(recieved_text) 

padding_type='post'
process = tokenizer.texts_to_sequences(recieved_text)
fakes_padded = pad_sequences(process, padding=padding_type, maxlen=max_length)           

classes = model.predict(fakes_padded)

#If class is close to 1, the probability of it being a spam is high

for x in range(len(recieved_text)):
  print(recieved_text[x])
  print(classes[x])

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: text, Length: 5572, dtype: object
175/175 [==============================] - 0s 2ms/step
Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
[3.3338958e-07]
Ok lar... Joking wif u oni...
[7.593771e-06]
Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receiv

[2.8168483e-06]
Check audrey's status right now
[0.00031088]
Busy here. Trying to finish for new year. I am looking forward to finally meeting you...
[7.1324206e-07]
Good afternoon sunshine! How dawns that day ? Are we refreshed and happy to be alive? Do we breathe in the air and smile ? I think of you, my love ... As always
[2.2328091e-08]
Well i know Z will take care of me. So no worries.
[5.09739e-08]
Update_Now - Xmas Offer! Latest Motorola, SonyEricsson & Nokia & FREE Bluetooth! Double Mins & 1000 Txt on Orange. Call MobileUpd8 on 08000839402 or call2optout/F4Q=
[0.9999832]
Here is your discount code RP176781. To stop further messages reply stop. www.regalportfolio.co.uk. Customer Services 08717205546
[0.99994075]
Wat uniform? In where get?
[2.6891864e-06]
Cool, text me when you're ready
[0.00061618]
Hello my boytoy ... Geeee I miss you already and I just woke up. I wish you were here in bed with me, cuddling me. I love you ...
[8.8391294e-10]
I will spoil you in bed as well :)
[3

[0.99981624]
Have a good evening! Ttyl
[0.00045651]
Hmm .. Bits and pieces lol ... *sighs* ...
[1.5906997e-05]
Hahaha..use your brain dear
[0.00142044]
Hey. You got any mail?
[8.054254e-08]
Sorry light turned green, I meant another friend wanted  &lt;#&gt;  worth but he may not be around
[1.8305367e-10]
Thanks for yesterday sir. You have been wonderful. Hope you enjoyed the burial. MojiBiola
[4.737287e-06]
U have a secret admirer. REVEAL who thinks U R So special. Call 09065174042. To opt out Reply REVEAL STOP. 1.50 per msg recd. Cust care 07821230901
[0.99997425]
Hi mate its RV did u hav a nice hol just a message 3 say hello coz havenåÕt sent u 1 in ages started driving so stay off roads!RVx
[5.227302e-07]
Dear Voucher Holder, To claim this weeks offer, at you PC please go to http://www.e-tlp.co.uk/expressoffer Ts&Cs apply. To stop texts, txt STOP to 80062
[0.99998444]
Thank you so much. When we skyped wit kz and sura, we didnt get the pleasure of your company. Hope you are good. We'v

[2.404132e-09]
Then Ì_ wait 4 me at bus stop aft ur lect lar. If i dun c Ì_ then i go get my car then come back n pick Ì_.
[3.3367512e-12]
Aight will do, thanks again for comin out
[1.527045e-05]
No..but heard abt tat..
[1.7417988e-05]
Please call our customer service representative on FREEPHONE 0808 145 4742 between 9am-11pm as you have WON a guaranteed å£1000 cash or å£5000 prize!
[0.9999027]
Yes..he is really great..bhaji told kallis best cricketer after sachin in world:).very tough to get out.
[3.1464634e-10]
&lt;#&gt;  am I think? Should say on syllabus
[3.118414e-09]
Umma. Did she say anything
[1.1738863e-06]
Give me a sec to think think about it
[6.793367e-08]
Panasonic & BluetoothHdset FREE. Nokia FREE. Motorola FREE & DoubleMins & DoubleTxt on Orange contract. Call MobileUpd8 on 08000839402 or call 2optout
[0.99992985]
I don't quite know what to do. I still can't get hold of anyone. I cud pick you up bout 7.30pm and we can see if they're in the pub?
[1.5453454e-11]
Poyyarikatu

[0.99991083]
Ya tel, wats ur problem..
[4.9489525e-05]
No. 1 Nokia Tone 4 ur mob every week! Just txt NOK to 87021. 1st Tone FREE ! so get txtin now and tell ur friends. 150p/tone. 16 reply HL 4info
[0.99994254]
i dnt wnt to tlk wid u
[3.5186745e-06]
We spend our days waiting for the ideal path to appear in front of us.. But what we forget is.. \paths are made by walking.. not by waiting..\" Goodnight!"
[0.00648473]
Its ok my arm is feeling weak cuz i got a shot so we can go another time
[1.2699762e-08]
Please reserve ticket on saturday eve from chennai to thirunelvali and again from tirunelvali to chennai on sunday eve...i already see in net..no ticket available..i want to book ticket through tackle ..
[0.00029413]
Storming msg: Wen u lift d phne, u say \HELLO\" Do u knw wt is d real meaning of HELLO?? . . . It's d name of a girl..! . . . Yes.. And u knw who is dat girl?? \"Margaret Hello\" She is d girlfrnd f Grahmbell who invnted telphone... . . . . Moral:One can 4get d name of a pe

...

[7.001142e-08]
Oh :-)only 4 outside players allowed to play know
[0.00046856]
I anything lor.
[1.7069094e-07]
Erutupalam thandiyachu
[0.01046134]
Y cant u try new invention to fly..i'm not joking.,
[0.00019272]
No..its ful of song lyrics..
[1.2019789e-05]
What do u reckon as need 2 arrange transport if u can't do it, thanks
[3.0740032e-06]
True lov n care wil nevr go unrecognized. though somone often makes mistakes when valuing it. but they will definitly undrstnd once when they start missing it.
[1.4997529e-06]
Shopping? Eh ger i toking abt syd leh...Haha
[1.2018778e-06]
What not under standing.
[0.00035062]
have * good weekend.
[0.00074595]
Miss call miss call khelate kintu opponenter miss call dhorte lage. Thats d rule. One with great phone receiving quality wins.
[0.00974815]
Call me when you get the chance plz &lt;3
[8.093652e-07]
The new deus ex game comin early next yr
[0.06908657]
My computer just fried the only essential part we don't keep spares of because my fucking idiot ro

[2.99384e-09]
Honey, can you pls find out how much they sell Predicte in Nigeria. And how many times can it be used. Its very important to have a reply before monday
[3.419519e-06]
E admin building there? I might b slightly earlier... I'll call u when i'm reaching...
[3.8720073e-06]
fyi I'm at usf now, swing by the room whenever
[0.00027699]
i can call in  &lt;#&gt;  min if thats ok
[1.3523126e-09]
Ummmmmaah Many many happy returns of d day my dear sweet heart.. HAPPY BIRTHDAY dear
[9.577876e-09]
ÌÏ no home work to do meh... 
[2.2691029e-07]
Anything is valuable in only 2 situations: First- Before getting it... Second- After loosing it...
[1.3698694e-05]
Me too. Mark is taking forever to pick up my prescription and the pain is coming back.
[1.5422945e-08]
How's ur paper?
[2.72719e-05]
Got smaller capacity one? Quite ex...
[9.334413e-05]
Check Out Choose Your Babe Videos @ sms.shsex.netUN fgkslpoPW fgkslpo
[0.00541664]
Im good! I have been thinking about you...
[2.3307605e-06]
u r a win

[8.316333e-11]
I'm already back home so no probably not
[4.4754094e-07]
Great News! Call FREEFONE 08006344447 to claim your guaranteed å£1000 CASH or å£2000 gift. Speak to a live operator NOW!
[0.9998003]
Hi this is Amy, we will be sending you a free phone number in a couple of days, which will give you an access to all the adult parties...
[0.00030606]
I am in bus on the way to calicut
[8.8717876e-08]
Hi its me you are probably having too much fun to get this message but i thought id txt u cos im bored! and james has been farting at me all night
[3.0744792e-07]
hi baby im sat on the bloody bus at the mo and i wont be home until about 7:30 wanna do somethin later? call me later ortxt back jess xx
[8.315596e-09]
Welcome to Select, an O2 service with added benefits. You can now call our specially trained advisors FREE from your mobile by dialling 402.
[0.9999368]
I lost 4 pounds since my doc visit last week woot woot! Now I'm gonna celebrate by stuffing my face!
[0.00075314]
U coming bac

I will be gentle baby! Soon you will be taking all  &lt;#&gt;  inches deep inside your tight pussy...
[9.593237e-08]
NOT MUCH NO FIGHTS. IT WAS A GOOD NITE!!
[2.7001624e-07]
Ok.ok ok..then..whats ur todays plan
[3.8892157e-08]
Nt joking seriously i told
[1.0657472e-05]
Watching ajith film ah?
[1.6700833e-05]
Ooooooh I forgot to tell u I can get on yoville on my phone
[1.3155126e-07]
All done, all handed in. Don't know if mega shop in asda counts as celebration but thats what i'm doing!
[7.77655e-08]
I dont know exactly could you ask chechi.
[6.0139877e-07]
Dunno lei shd b driving lor cos i go sch 1 hr oni.
[2.3053208e-06]
As in i want custom officer discount oh.
[3.0167012e-05]
That's necessarily respectful
[0.00017394]
Hi. Hope you had a good day. Have a better night.
[1.32881805e-05]
And he's apparently bffs with carly quick now
[0.04233609]
HARD BUT TRUE: How much you show &amp;  express your love to someone....that much it will hurt when they leave you or you get seperated...!ö´Ó

[0.9998285]
No, I was trying it all weekend ;V
[3.1311864e-09]
You know, wot people wear. T shirts, jumpers, hat, belt, is all we know. We r at Cribbs
[4.906982e-05]
Cool, what time you think you can get here?
[5.2146524e-06]
Wen did you get so spiritual and deep. That's great
[1.3648222e-06]
Have a safe trip to Nigeria. Wish you happiness and very soon company to share moments with
[0.00011124]
Hahaha..use your brain dear
[0.00142044]
Well keep in mind I've only got enough gas for one more round trip barring a sudden influx of cash
[1.3815075e-05]
Yeh. Indians was nice. Tho it did kane me off a bit he he. We shud go out 4 a drink sometime soon. Mite hav 2 go 2 da works 4 a laugh soon. Love Pete x x
[1.2545467e-11]
Yes i have. So that's why u texted. Pshew...missing you so much
[2.0726885e-07]
No. I meant the calculation is the same. That  &lt;#&gt; units at  &lt;#&gt; . This school is really expensive. Have you started practicing your accent. Because its important. And have you decide

In [21]:
#importing output csv file

output_data = pd.read_csv('output_spam.csv')

In [22]:
#check for NaN or Unnamed values

output_data.isna().sum()

Unnamed: 0    0
text          0
dtype: int64

In [23]:
output_data = output_data.drop(['Unnamed: 0'], axis=1)

In [24]:
#setting data to array form

text = np.asanyarray(output_data['text'])

In [25]:
new_classes = ['spam' if x > 0.1 else 'ham' for x in classes]

In [26]:
#putting spam or ham in the output csv file

prediction = pd.DataFrame(new_classes, columns = ['text']).to_csv('Borbe_spam_output.csv')